In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re
from tqdm import tqdm
import uuid

from datasets import Dataset

### parse card

In [6]:
url = "https://www.povarenok.ru/recipes/~2/?sort=rating&order=desc"
response = requests.get(url=url)
soup = BeautifulSoup(response.text)
items = soup.find_all("article", class_="item-bl")
len(items)

15

In [17]:
card = items[0]

title = card.find("h2").text.strip()
title

'Жареный картофель "Хозяйка отдыхает"'

In [24]:
link = card.find("h2").find("a")["href"]
link

'https://www.povarenok.ru/recipes/show/54138/'

In [18]:
ingridients = [item.text for item in card.find("span", class_="list").find_all("a")]
ingridients

['Картофель',
 'Морковь',
 'Лук репчатый',
 'Помидор',
 'Приправа',
 'Масло растительное',
 'Масло оливковое',
 'Зелень',
 'Чеснок']

In [21]:
views = int(card.find("span", class_="i-views").text)
views

2639094

In [22]:
likes = int(card.find("span", class_="i-likes").text)
likes

20834

In [23]:
ups = int(card.find("span", class_="i-up").text)
ups

2554

### parse page

In [45]:
url = "https://www.povarenok.ru/recipes/show/130312/"
response = requests.get(url=url)
soup = BeautifulSoup(response.text)
title_receipt = soup.find_all("h2")[2].text
title_receipt

'Рецепт «Салат "Архиерейский" с мясом птицы»:'

In [46]:
items = soup.find_all("li", class_="cooking-bl")
items = [item.text.strip() for item in items]
items

['Ингредиенты для приготовления салата.',
 'Лук репчатый нашинковать полукольцами.',
 'На сковороду добавить растительное масло (2 ст. ложки), выложить лук и легка припустить на слабом огне до прозрачности. Затем наклонить сковороду и слить масло. В салате оно не нужно.',
 'Сырую морковь нашинковать на корейской терке.',
 'Филе индейки отварить, порезать брусочками.',
 'Яйца отварить, порезать брусочками.',
 'Редьку натереть на корейской терке.',
 'Ингредиенты соединить.',
 'Добавить соль по вкусу. Заправить салат майонезом.',
 'Приятного аппетита!']

In [47]:
full_receipt_text = [f"{i+1}. {item}" for i, item in enumerate(items)]
full_receipt_text = "\n".join(full_receipt_text)
full_receipt_text = f"{title_receipt}\n{full_receipt_text}"
print(full_receipt_text)

Рецепт «Салат "Архиерейский" с мясом птицы»:
1. Ингредиенты для приготовления салата.
2. Лук репчатый нашинковать полукольцами.
3. На сковороду добавить растительное масло (2 ст. ложки), выложить лук и легка припустить на слабом огне до прозрачности. Затем наклонить сковороду и слить масло. В салате оно не нужно.
4. Сырую морковь нашинковать на корейской терке.
5. Филе индейки отварить, порезать брусочками.
6. Яйца отварить, порезать брусочками.
7. Редьку натереть на корейской терке.
8. Ингредиенты соединить.
9. Добавить соль по вкусу. Заправить салат майонезом.
10. Приятного аппетита!


In [ ]:
from datasets import load_dataset
dataset = load_dataset("dim/povarenok_links")
dataset = dataset['train']
dataset['link'][:100]

In [55]:
dataset.to_list()[0]

{'title': 'Котлеты "Ласточкино гнездо"',
 'ingridients': ['Фарш мясной',
  'Хлеб',
  'Чеснок',
  'Укроп',
  'Соль',
  'Перец черный',
  'Яйцо куриное',
  'Майонез',
  'Кетчуп',
  'Лук репчатый',
  'Помидор',
  'Сыр твердый',
  'Перец болгарский'],
 'views': 6639388,
 'likes': 33991,
 'ups': 4683,
 'link': 'https://www.povarenok.ru/recipes/show/5659/'}

In [1]:
from datasets import load_dataset, Dataset, concatenate_datasets

dataset = load_dataset("dim/povarenok")
dataset = dataset["train"]
dataset

/home/kosenko/miniconda3/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Dataset({
    features: ['full_receipt_text', 'steps', 'title_receipt', 'title', 'ingridients', 'views', 'likes', 'ups', 'link'],
    num_rows: 46500
})

In [4]:
dataset[0]

{'full_receipt_text': 'Рецепт «Котлеты "Ласточкино гнездо"»:\n1. К фаршу добавить соль, перец, приправы к мясу, мелко нарезанные чеснок и укроп, замоченный в молоке или воде и перекрученный на мясорубке батон, 2 яйца, \r\nхорошо вымесить и сформировать лепешки (у меня выходит 12 штук). \r\nВыложить их на смазанный растительным маслом противень.\n2. Нарезать сыр, лук, помидор, перец (у меня сегодня его не было) тонкими ломтиками (кружочками). \r\nНа мясную лепешку выкладываем: кетчуп - лук - майонез - помидор - майонез - сыр. Кружочек перца вдавливаем вокруг начинки и саму начинку тоже легонько внутрь примять. В разогретую духовку - 180*С 30 мин.\n3. Приятного аппетита!!!',
 'steps': ['К фаршу добавить соль, перец, приправы к мясу, мелко нарезанные чеснок и укроп, замоченный в молоке или воде и перекрученный на мясорубке батон, 2 яйца, \r\nхорошо вымесить и сформировать лепешки (у меня выходит 12 штук). \r\nВыложить их на смазанный растительным маслом противень.',
  'Нарезать сыр, лук, 

In [5]:
", ".join(dataset[0]['ingridients'])

'Фарш мясной, Хлеб, Чеснок, Укроп, Соль, Перец черный, Яйцо куриное, Майонез, Кетчуп, Лук репчатый, Помидор, Сыр твердый, Перец болгарский'

In [2]:
dataset = dataset.sort(column_names=['views'], reverse=True)

In [3]:
dataset[0]

{'full_receipt_text': 'Рецепт «Оладьи на кефире "Идеальные"»:\n1. Кефир автор советует достать из холодильника заранее, чтобы был комнатной температуры.\n\r\nИтак, кефир вылить в миску. Добавить яйцо, соль, сахар. Перемешать. Я пользовалась миксером.\n2. Затем добавить просеянную муку и соду.\r\nОбычно соду бросают в кефир, чтобы она погасилась, здесь же нужно добавлять именно с мукой.\n3. Тщательно перемешать.\r\n Размешивать тесто нужно сразу хорошо, потому что потом его мешать категорически нельзя!\r\nИ оставить тесто на полчаса, чтобы пошла реакция кефира с содой.\n4. Спустя полчаса на поверхности теста образовались пузырьки (реакция пошла), можно приступать к жарке.\n5. Нагреть растительное масло.\r\nПомним, что тесто больше нельзя мешать.\n\r\nТесто берем ложкой с краю, стараясь не беспокоить тесто, и как бы отщипываем об стенку миски. Выкладываем аккуратно на сковородку и обжариваем на среднем огне.\n\r\nКогда на поверхности начнут появляться и лопаться вот такие пузырьки (см. ф

In [4]:
dataset = dataset.select(range(10_000))
dataset.push_to_hub("dim/povarenok_10k")

Pushing dataset shards to the dataset hub: 100%|██████████| 1/1 [08:35<00:00, 515.85s/it]
